# Solving cryptarithmetic puzzle - SEND + MORE = MONEY

In [1]:
from itertools import permutations

In [2]:
letters = 'SENDMORY'
perms = list(permutations(range(10), len(letters)))
print(f"We have {len(perms)} permutations to check")
# first 5
for p in perms[:5]:
    print(p)

We have 1814400 permutations to check
(0, 1, 2, 3, 4, 5, 6, 7)
(0, 1, 2, 3, 4, 5, 6, 8)
(0, 1, 2, 3, 4, 5, 6, 9)
(0, 1, 2, 3, 4, 5, 7, 6)
(0, 1, 2, 3, 4, 5, 7, 8)


In [4]:
# worst cryptoarithmetic puzzle would be 10 out of 10
perms = list(permutations(range(10), 10))
print(f"We have {len(perms)} permutations to check")
# first 5
for p in perms[:5]:
    print(p)

We have 3628800 permutations to check
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
(0, 1, 2, 3, 4, 5, 6, 7, 9, 8)
(0, 1, 2, 3, 4, 5, 6, 8, 7, 9)
(0, 1, 2, 3, 4, 5, 6, 8, 9, 7)
(0, 1, 2, 3, 4, 5, 6, 9, 7, 8)


In [5]:
# so selection 10 out of 10 is same as
# 10!
import math
print(math.factorial(10))

3628800


<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/739752f4180752db6442c9d8211de651513e594a" width="200">

Src: https://en.wikipedia.org/wiki/Permutation

In [10]:
# Solving cryptarithmetic puzzle - SEND + MORE = MONEY

## Niks solution

from itertools import permutations

#With the help of ChatGPT this is what the solution could look like
#I provided the main idea and the restrictions

def solve_send_more_money():
    #SEND + MORE = MONEY
    # So all the letters used in the puzzle are:
    letters = 'SENDMORY'

    # Generate all permutations of digits for the letters
    for perm in permutations(range(10), len(letters)):

        # Map the permutation of digits to the letters
        solution = dict(zip(letters, perm))

        # let's keep only those solutions where M is 1
        if solution['M'] != 1: # M has to be one by the restrictions of the puzzle
            # SEND + MORE = MONEY so M has to be 1
            continue

        # Ensure S and M are not 0 as they are the first letters of the numbers
        if solution['S'] == 0: 
            # or solution['M'] == 0: # We could take this out as we already know M is 1!
            continue

        # Construct the numbers from the letters
        send = solution['S']*1000 + solution['E']*100 + solution['N']*10 + solution['D']
        more = solution['M']*1000 + solution['O']*100 + solution['R']*10 + solution['E']
        money = solution['M']*10000 + solution['O']*1000 + solution['N']*100 + solution['E']*10 + solution['Y']

        # Check if the sum is correct
        if send + more == money:
            return solution  # Return the solution if the condition is satisfied

    return "No solution exists."

# if __name__ == "__main__":
#     solution = solve_send_more_money()
#     print("Solution:")
#     for letter, digit in solution.items():
#         print(f"{letter} = {digit}")

In [7]:
solve_send_more_money() # so almost brute force with minimal constraints works well enough

{'S': 9, 'E': 5, 'N': 6, 'D': 7, 'M': 1, 'O': 0, 'R': 8, 'Y': 2}

In [11]:
solve_send_more_money()

{'S': 9, 'E': 5, 'N': 6, 'D': 7, 'M': 1, 'O': 0, 'R': 8, 'Y': 2}

In [12]:
%%timeit
solve_send_more_money()

1.58 s ± 25.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## General approach

1. Generate all permutations of the digits 0-9
2. For each permutation, check if the equation SUMMABLE1 + SUMMABLE2 = TOTAL is satisfied
3. If the equation is satisfied, print the permutation

### Optimizations

1. Skip permutations where the first digit of SUMMABLE1 or SUMMABLE2 is 0
2. Skip permutations where the first digit of TOTAL is 0

In [14]:
def solve_summables(summable1:str, summable2:str, total:str) -> dict[str, int]:
    letters = set(summable1 + summable2 + total)
    for perm in permutations(range(10), len(letters)):
        solution = dict(zip(letters, perm))
        if solution[summable1[0]] == 0 or solution[summable2[0]] == 0 or solution[total[0]] == 0:
            continue
        # now we check if the sum is correct
        if sum(int(''.join(str(solution[letter]) for letter in word)) for word in (summable1, summable2)) == int(''.join(str(solution[letter]) for letter in total)):
            return solution
    return {"No solution exists.":0} # or None or "No solution exists." or whatever

In [15]:
# test with SEND + MORE = MONEY
solve_summables('SEND', 'MORE', 'MONEY')

{'N': 6, 'Y': 2, 'R': 8, 'M': 1, 'D': 7, 'S': 9, 'O': 0, 'E': 5}

In [16]:
# ZEROES + ONES = BINARY
solve_summables('ZEROES', 'ONES', 'BINARY')

{'I': 0,
 'N': 1,
 'A': 5,
 'Y': 4,
 'R': 8,
 'Z': 6,
 'B': 7,
 'S': 2,
 'O': 3,
 'E': 9}

In [25]:
# let's create a pretty print funciton to show solutions
def pretty_print(solution:dict[str, int], summable1, summable2, total):
    for word in (summable1, summable2):
        digits = ''.join(str(solution[letter]) for letter in word)
        # we to right justify the digits to align them
        print(f"{digits:>{len(total)}}")
        # print(f"{digits}", end='+\n')
    print(''.join(str(solution[letter]) for letter in total))

In [26]:
pretty_print(solve_summables('ZEROES', 'ONES', 'BINARY'), 'ZEROES', 'ONES', 'BINARY')

698392
  3192
701584


In [27]:
# now let's make a generic solver that takes any number of summables
def solve_summables(*args) -> dict[str, int]:
    letters = set(''.join(args))
    for perm in permutations(range(10), len(letters)):
        solution = dict(zip(letters, perm))
        if any(solution[word[0]] == 0 for word in args):
            continue
        if sum(int(''.join(str(solution[letter]) for letter in word)) for word in args[:-1]) == int(''.join(str(solution[letter]) for letter in args[-1])):
            return solution
    return {"No solution exists.":0} # or None or "No solution exists." or whatever

In [28]:
solve_summables('ZEROES', 'ONES', 'BINARY')

{'I': 0,
 'N': 1,
 'A': 5,
 'Y': 4,
 'R': 8,
 'Z': 6,
 'B': 7,
 'S': 2,
 'O': 3,
 'E': 9}

In [29]:
# above is fine but let's change our function signature to list[str] and str returning dict[str, int] or None
def solve_summables(words:list[str], total:str) -> dict[str, int]:
    letters = set(''.join(words) + total)
    for perm in permutations(range(10), len(letters)):
        solution = dict(zip(letters, perm))
        if any(solution[word[0]] == 0 for word in words + [total]):
        # so any is similar to existential quantifier in logic
            continue
        if sum(int(''.join(str(solution[letter]) for letter in word)) for word in words) == int(''.join(str(solution[letter]) for letter in total)):
            return solution
    return None

In [30]:
# test with SEND + MORE = MONEY
solve_summables(['SEND', 'MORE'], 'MONEY')

{'N': 6, 'Y': 2, 'R': 8, 'M': 1, 'D': 7, 'S': 9, 'O': 0, 'E': 5}

In [31]:
# test with ZEROES + ONES = BINARY
solve_summables(['ZEROES', 'ONES'], 'BINARY')

{'I': 0,
 'N': 1,
 'A': 5,
 'Y': 4,
 'R': 8,
 'Z': 6,
 'B': 7,
 'S': 2,
 'O': 3,
 'E': 9}

In [32]:
# let's test with something with 3 or more summables
solve_summables(['ZEROES', 'ONES', 'TWOS'], 'BINARY')

In [33]:
solve_summables(['ONE','ONE','ONE', 'ONE'], 'FOUR')

{'N': 2, 'U': 0, 'F': 1, 'R': 4, 'O': 3, 'E': 6}

In [34]:
solve_summables(['ONE','ONE','ONE', 'ONE'], 'TEN')

{'N': 8, 'T': 7, 'O': 1, 'E': 2}

In [35]:
# TODO add generic pretty print function for any number of summables
def pretty_print(solution:dict[str, int], words:list[str], total:str):
    width = max(len(word) for word in words + [total])
    for word in words:
        digits = ''.join(str(solution[letter]) for letter in word)
        print(f"{digits:>{width}}")
    print(''.join(str(solution[letter]) for letter in total))

In [36]:
# test with ONE + ONE + ONE + ONE = TEN
pretty_print(solve_summables(['ONE','ONE','ONE', 'ONE'], 'TEN'), ['ONE','ONE','ONE', 'ONE'], 'TEN')

182
182
182
182
728
